# Bin Packing

In a bin packing problem, we are given the set of items $N = \{ 0, ..., n - 1 \}$ and bins with capacity $c$.
Each item $i \in N$ has the weight $w_i$.
We want to pack the items into bins while minimizing the number of bins.

## DP Formulation

Consider using bins one by one and packing items one by one.
Let $U$ be the set of unpacked items, and $r$ be the remaining capacity of the current bin.
Item $i$ can be packed into the current bin only if $w_i \leq r$.
After packing $i$, $U$ becomes $U \setminus \{ i \}$ and $r$ becomes $r - w_i$.

When no items fit into the current bin, we need to open a new bin.
We can select an arbitrary item as the first item in the new bin without loosing the optimality.

It is known that there is a solution that packs item $i$ in the $i$ th bin or earlier.
Let $k$ be the number of used bins (including the current bin).
Then, we only need to consider items $i$ in $U$ with $i + 1 \geq k$.
We have the following DP model:

$$
\begin{align}
    \text{compute } & V(N, 0, 0) \\
    & V(U, r, k) = \begin{cases}
        \min\limits_{i \in U : w_i \leq r \land i + 1 \geq k} V(U \setminus \{ i \}, r - w_i, k) & \text{if } \exists i \in U, w_i \leq r \land i + 1 \geq k \\
        1 + V(U \setminus \{ i \}, r, k + 1) & \text{else if } \exists i \in U, i \geq k \\
        0 & \text{else if } U = \emptyset \\
        \infty & \text{otherwise.}
    \end{cases}.
\end{align}
$$

If two states $(U, r, k)$ and $(U, r', k')$ have the same set of unpacked items, if $r \geq r'$ and $k \leq k'$, $(U, r, k)$ leads to a better solution. Therefore,

$$
    V(U, r, k) \leq V(U, r', k') \text{ if } r \geq r' \land k \leq k'.
$$

If we ignore the fact that an item cannot be divided for multiple bin, we get the following lower bound:

$$
    V(U, r, k) \geq \left\lceil \frac{\sum_{i \in U} w_i - r}{c} \right\rceil.
$$

Consider only items $i$ with $w_i \geq \frac{c}{2}$.
Each bin contains at most one of bins $i$ with $w_i > \frac{c}{2}$.
Similarly, each bin contains at most two bins with $w_i = \frac{c}{2}$, and such items are not packed in the same bin having items with $t_i > \frac{c}{2}$.
If $r \geq \frac{c}{2}$, we can possibly use the current bin to pack the items.
Therefore,

$$
    V(U, r, k) \geq \begin{cases}
        \sum\limits_{ i \in U : w_i > \frac{c}{2} } 1 + \left\lceil \sum\limits_{i \in U : w_i = \frac{c}{2}} \frac{1}{2} \right\rceil & \text{if } r < \frac{c}{2} \\
        \sum\limits_{ i \in U : w_i > \frac{c}{2} } 1 + \left\lceil \sum\limits_{i \in U : w_i = \frac{c}{2}} \frac{1}{2} \right\rceil - 1 & \text{if } r \geq \frac{c}{2}.
    \end{cases}
$$

Similaly, if we consider only items $i$ with $w_i \geq \frac{c}{3}$, we get the following bound:

$$
    V(U, r, k) \geq \begin{cases}
        \left\lceil \sum\limits_{ i \in U : w_i > \frac{2c}{3} } 1 + \sum\limits_{i \in U : w_i = \frac{2c}{3}} \frac{2}{3} + \sum\limits_{i \in U : w_i \in (\frac{c}{3}. \frac{2c}{3})} \frac{1}{2} + \sum\limits_{i \in U : w_i = \frac{c}{3}} \frac{1}{3} \right\rceil & \text{if } r < \frac{c}{3} \\
        \left\lceil \sum\limits_{ i \in U : w_i > \frac{2c}{3} } 1 + \sum\limits_{i \in U : w_i = \frac{2c}{3}} \frac{2}{3} + \sum\limits_{i \in U : w_i \in (\frac{c}{3}. \frac{2c}{3})} \frac{1}{2} + \sum\limits_{i \in U : w_i = \frac{c}{3}} \frac{1}{3} \right\rceil - 1 & \text{if } r \geq \frac{c}{3} \\
    \end{cases}
$$

## Install DIDPPy

In [1]:
!pip install didppy

## Data

In [2]:
# Number of items
n = 5
# Capacity
c = 5
# Weights
w = [2, 2, 1, 3, 2]

## Preprocessing

In [3]:
# The weight in the first term of the second bound.
weight_2_1 = [1 if w[i] > c / 2 else 0 for i in range(n)]
# The weight in the second term of the second bound.
weight_2_2 = [1 / 2 if w[i] == c / 2 else 0 for i in range(n)]
# The weight in the third bound (truncated to three decimal points).
weight_3 = [
    1.0
    if w[i] > c * 2 / 3
    else 2 / 3 // 0.001 * 1000
    if w[i] == c * 2 / 3
    else 0.5
    if w[i] > c / 3
    else 1 / 3 // 0.001 * 1000
    if w[i] == c / 3
    else 0.0
    for i in range(n)
]

## Modeling

In [4]:
import math

import didppy as dp

model = dp.Model()

item = model.add_object_type(number=n)

# U
unpacked = model.add_set_var(object_type=item, target=list(range(n)))
# r
remaining = model.add_int_resource_var(target=0, less_is_better=False)
# k (we want to compare the number of bins with the index of an item) 
number_of_bins = model.add_element_resource_var(
    object_type=item,
    target=0,
    less_is_better=True,
)

weight = model.add_int_table(w)

for i in range(n):
    pack = dp.Transition(
        name="pack {}".format(i),
        cost=dp.IntExpr.state_cost(),
        effects=[
            (unpacked, unpacked.remove(i)),
            (remaining, remaining - weight[i]),
        ],
        preconditions=[
            unpacked.contains(i),
            weight[i] <= remaining,
            i + 1 >= number_of_bins,
        ],
    )
    model.add_transition(pack)
    
for i in range(n):
    open_new = dp.Transition(
        name="open a new bin and pack {}".format(i),
        cost=1 + dp.IntExpr.state_cost(),
        effects=[
            (unpacked, unpacked.remove(i)),
            (remaining, c - weight[i]),
            (number_of_bins, number_of_bins + 1)
        ],
        preconditions=[
            unpacked.contains(i),
            i >= number_of_bins,
            weight[i] > remaining,
        ]
        + [
            ~unpacked.contains(j) | (weight[j] > remaining)
            for j in range(n)
            if i != j
        ],
    )
    model.add_transition(open_new, forced=True)

model.add_base_case([unpacked.is_empty()])

model.add_dual_bound(
    math.ceil((weight[unpacked] - remaining) / c)
)

weight_2_1_table = model.add_int_table(weight_2_1)
weight_2_2_table = model.add_float_table(weight_2_2)
model.add_dual_bound(
    weight_2_1_table[unpacked]
    + math.ceil(weight_2_2_table[unpacked])
    - (remaining >= c / 2).if_then_else(1, 0)
)

weight_3_table = model.add_float_table(weight_3)
model.add_dual_bound(
    math.ceil(weight_3_table[unpacked])
    - (remaining >= c / 3).if_then_else(1, 0)
)

## Solving

In [5]:
solver = dp.CABS(model, quiet=True)
solution = solver.search()

print("Transitions to apply:")
print("")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))

Transitions to apply:

open a new bin and pack 0
pack 1
pack 2
open a new bin and pack 3
pack 4

Cost: 2
